In [2]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb

In [3]:
df= pd.read_excel("../data/raw/entrenamiento.xlsx")
df= df.rename(columns={"C7.1": "C8", "des": "QUALITY"})

In [4]:
# Filtrar las filas sin datos faltantes
df_no_missing = df.dropna()

# Separar este conjunto filtrado para la validación
X_validate = df_no_missing.drop("QUALITY", axis=1)
y_validate = df_no_missing["QUALITY"]

# Eliminar estas filas del conjunto original para obtener el conjunto de entrenamiento
df_train = df.drop(df_no_missing.index)


In [5]:
def impute_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identificar outliers
    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)

    # Imputar outliers con la mediana de la columna
    data.loc[outliers, column] = data[column].median()
    return data

for column in df_train.columns:
    if column != "QUALITY" and df_train[column].dtype != 'object':
        df_train = impute_outliers_iqr(df_train, column)


In [6]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Separar las características y el objetivo en el conjunto de entrenamiento
X_train = df_train.drop("QUALITY", axis=1)
y_train = df_train["QUALITY"]

# Estandarizar las características (tanto para el conjunto de entrenamiento como para el de validación)
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_validate_standardized = scaler.transform(X_validate)

# Entrenar el modelo XGBoost con los hiperparámetros que discutimos anteriormente
best_params = {
    'subsample': 0.8, 
    'n_estimators': 200, 
    'min_child_weight': 1, 
    'max_depth': 8, 
    'learning_rate': 0.1, 
    'gamma': 0, 
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror', 
    'random_state': 42
}
model = XGBRegressor(**best_params)
model.fit(X_train_standardized, y_train)


/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [7]:
# Realizar predicciones en el conjunto de validación
validate_predictions = model.predict(X_validate_standardized)

# Calcular el MAE en el conjunto de validación
mae_validate = mean_absolute_error(y_validate, validate_predictions)
print(f"Mean Absolute Error (MAE) en el conjunto de validación: {mae_validate:.4f}")


Mean Absolute Error (MAE) en el conjunto de validación: 0.5438


In [8]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Load the data and preprocess
df = pd.read_excel("../data/raw/entrenamiento.xlsx")
df = df.rename(columns={"C7.1": "C8", "des": "QUALITY"})

# Handle the specific outlier case
high_values = df["C8"] > 900
df.loc[high_values, "C8"] = df.loc[high_values, "C8"] / 1000

# Function to impute outliers using IQR and column median
def impute_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)

    # Impute outliers with the median of the column
    data.loc[outliers, column] = data[column].median()
    return data

# Impute outliers using IQR method
for column in df.columns:
    if df[column].dtype != 'object':  # Apply only for numerical columns
        df = impute_outliers_iqr(df, column)

# Separate features and target
X = df.drop("QUALITY", axis=1)
y = df["QUALITY"]

# Standardize the data
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

# Define the model with best parameters
best_params = {
    'subsample': 0.8, 
    'n_estimators': 200, 
    'min_child_weight': 1, 
    'max_depth': 8, 
    'learning_rate': 0.1, 
    'gamma': 0, 
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror', 
    'random_state': 42
}

model = xgb.XGBRegressor(**best_params)

# Define the MAE scorer
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Perform 5-fold cross-validation
mae_scores = cross_val_score(model, X_standardized, y, cv=5, scoring=mae_scorer)

# Print the results
print("MAE scores from cross-validation:", -mae_scores)
print("Mean MAE score:", -mae_scores.mean())
print("Standard Deviation of MAE scores:", mae_scores.std())


/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/alberte/Desktop/Fruit/.venv/lib/python3.10/sit

MAE scores from cross-validation: [0.46915756 0.42916445 0.45710028 0.46035299 0.45224953]
Mean MAE score: 0.4536049602209868
Standard Deviation of MAE scores: 0.01340755409305219
